# CS-351
# SUPER RESOLUTION PROJECT

1) Muhammad Omer Bin Atique 2020335
2) Syed Ammar Bin Farrukh 2020468
3) Sameer Arif Khan 2020430

## Import All Libraries For Image Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import random as rand
import seaborn as sns
import scipy as sci
from sklearn.model_selection import train_test_split
import os
import cv2
from math import log10, sqrt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
import mtcnn
from PIL import Image
from scipy import ndimage
from tqdm import tqdm
from sklearn.datasets import fetch_lfw_people
from numpy.linalg import norm
import numpy as np
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split

2023-05-20 11:06:47.120468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 11:06:47.644224: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-20 11:06:47.714861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omer_atique/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-20 1

## Get Folder Names

In [3]:
dataset_path = "./dataset/"
groundtruth_images = []
image_size = 512
patch_size = 64

subfolders = sorted([f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))])

print(subfolders)

['balloons_ms', 'beads_ms', 'cd_ms', 'chart_and_stuffed_toy_ms', 'clay_ms', 'cloth_ms', 'egyptian_statue_ms', 'face_ms', 'fake_and_real_beers_ms', 'fake_and_real_food_ms', 'fake_and_real_lemon_slices_ms', 'fake_and_real_lemons_ms', 'fake_and_real_peppers_ms', 'fake_and_real_strawberries_ms', 'fake_and_real_sushi_ms', 'fake_and_real_tomatoes_ms', 'feathers_ms', 'flowers_ms', 'glass_tiles_ms', 'hairs_ms', 'jelly_beans_ms', 'oil_painting_ms', 'paints_ms', 'photo_and_face_ms', 'pompoms_ms', 'real_and_fake_apples_ms', 'real_and_fake_peppers_ms', 'sponges_ms', 'stuffed_toys_ms', 'superballs_ms', 'thread_spools_ms', 'watercolors_ms']


## Import All Images

In [4]:
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    print(subfolder_path)

    hyperspectral_image = np.empty((image_size, image_size, 31), dtype=np.uint8)
    i = 0
    for filename in sorted(os.listdir(subfolder_path)):
        if filename.endswith(".png"):
            image_file = filename
            image_path = os.path.join(subfolder_path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            hyperspectral_image[:, :, i] = image
            i = i + 1
 
    for y in range(0, image_size - patch_size + 1, stride):
        for x in range(0, image_size - patch_size + 1, stride):
            patch = hyperspectral_image[y:y+patch_size, x:x+patch_size, :]
            groundtruth_images.append(patch)

groundtruth_arrays = np.array(groundtruth_images)

print("Ground Truth Hyperspectral Images Shape:", groundtruth_arrays.shape)

./dataset/balloons_ms
./dataset/beads_ms
./dataset/cd_ms
./dataset/chart_and_stuffed_toy_ms
./dataset/clay_ms
./dataset/cloth_ms
./dataset/egyptian_statue_ms
./dataset/face_ms
./dataset/fake_and_real_beers_ms
./dataset/fake_and_real_food_ms
./dataset/fake_and_real_lemon_slices_ms
./dataset/fake_and_real_lemons_ms
./dataset/fake_and_real_peppers_ms
./dataset/fake_and_real_strawberries_ms
./dataset/fake_and_real_sushi_ms
./dataset/fake_and_real_tomatoes_ms
./dataset/feathers_ms
./dataset/flowers_ms
./dataset/glass_tiles_ms
./dataset/hairs_ms
./dataset/jelly_beans_ms
./dataset/oil_painting_ms
./dataset/paints_ms
./dataset/photo_and_face_ms
./dataset/pompoms_ms
./dataset/real_and_fake_apples_ms
./dataset/real_and_fake_peppers_ms
./dataset/sponges_ms
./dataset/stuffed_toys_ms
./dataset/superballs_ms
./dataset/thread_spools_ms
./dataset/watercolors_ms
Ground Truth Hyperspectral Images Shape: (7200, 64, 64, 31)


## Obtain A Low Resolution HSI Image of Shape (8,8,31)
One will simulate an image with low spatial dimensions and high spectral dimension. You'll obtain this by applying a 8,8 averaging filter on each band of your image of resolution 64,64,31 to obtain an image of resolution of 8,8,31. Let's call this one a low(spatial) resolution HSIimage.

In [5]:
import numpy as np
from scipy.ndimage import uniform_filter

filter_size = (8, 8)

lowres_hsi_images = np.zeros((groundtruth_arrays.shape[0], 8, 8, 31))
for i in range(groundtruth_arrays.shape[0]):
    for j in range(groundtruth_arrays.shape[3]):
        band = groundtruth_arrays[i, :, :, j]
        lowres_band = uniform_filter(band, size=filter_size)
        lowres_hsi_images[i, :, :, j] = lowres_band[::8, ::8]

In [6]:
print("Low-Resolution HSI Images Shape:", lowres_hsi_images.shape)

Low-Resolution HSI Images Shape: (7200, 8, 8, 31)


In [7]:
highres_rgb_images = np.zeros((groundtruth_arrays.shape[0], 64, 64, 3))
for i in range(groundtruth_arrays.shape[0]):
    for j in range(3):
        start_band = j * 10
        end_band = (j + 1) * 10
        rgb_band_avg = np.mean(groundtruth_arrays[i, :, :, start_band:end_band], axis=-1)
        highres_rgb_images[i, :, :, j] = rgb_band_avg

In [9]:

print("High-Resolution RGB Images Shape:", highres_rgb_images.shape)

High-Resolution RGB Images Shape: (7200, 64, 64, 3)


## Split Images To Form A Test And Train Set

In [10]:
train_ratio = 0.8
num_samples = groundtruth_arrays.shape[0]
num_train_samples = int(train_ratio * num_samples)
train_lowres_hsi = lowres_hsi_images[:num_train_samples]
train_highres_rgb = highres_rgb_images[:num_train_samples]
test_lowres_hsi = lowres_hsi_images[num_train_samples:]
test_highres_rgb = highres_rgb_images[num_train_samples:]
y_train = groundtruth_arrays[:num_train_samples]
y_test = groundtruth_arrays[num_train_samples:]

In [ ]:

train_lowres_hsi = train_lowres_hsi / 255.0
train_highres_rgb = train_highres_rgb / 255.0
test_lowres_hsi = test_lowres_hsi / 255.0
test_highres_rgb = test_highres_rgb / 255.0

In [13]:
from keras.layers import Input, Concatenate, Conv2D, Conv2DTranspose, BatchNormalization, Activation, LeakyReLU
from keras.models import Model

def create_super_resolution_model(input_shape_hr, input_shape_lr, output_shape):
    hr_input = Input(shape=input_shape_hr)
    
    lr_input = Input(shape=input_shape_lr)

    upsampled_lr = Conv2DTranspose(31, (8, 8), strides=(8, 8), padding='same')(lr_input)

    concat = Concatenate()([hr_input, upsampled_lr])

    x = Conv2D(64, (3, 3), strides=1, padding='same')(concat)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (3, 3), strides=1, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (3, 3), strides=1, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    
    x = Conv2DTranspose(128, (3, 3), strides=1, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2DTranspose(64, (3, 3), strides=1, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    output = Conv2D(31, (3, 3), strides=1, padding='same', activation='sigmoid')(x)

    model = Model(inputs=[hr_input, lr_input], outputs=output)
    return model

input_shape_hr = (64, 64, 3)
input_shape_lr = (8, 8, 31)
output_shape = (64, 64, 31)

model = create_super_resolution_model(input_shape_hr, input_shape_lr, output_shape)

print(model.summary())

2023-05-20 11:08:40.143360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 11:08:40.144552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omer_atique/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-20 11:08:40.144854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omer_atique/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-20 11:08:40.145033: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 8, 31)]   0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 64, 64, 31)  61535       ['input_2[0][0]']                
 ose)                                                                                             
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 34)   0           ['input_1[0][0]',            

In [ ]:
model.compile(optimizer='adam', loss='mse') 
model.fit([train_highres_rgb, train_lowres_hsi], y_train, validation_data=([test_highres_rgb, test_lowres_hsi], y_test), epochs=10, batch_size=32)

Epoch 1/10
180/180 [==============================] - 982s 5s/step - loss: 0.0084 - val_loss: 0.0000e+00
Epoch 2/10
180/180 [==============================] - 931s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
180/180 [==============================] - 931s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
180/180 [==============================] - 1291s 7s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
 20/180 [==>...........................] - ETA: 12:52 - loss: 0.0000e+00